In [ ]:
# default_exp odds_ratio

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import numpy as np
from corradin_ovp_utils.catalog import test_data_catalog, conf_test_data_catalog
from fastcore.test import ExceptionExpected

In [ ]:
#export
import pandas as pd
from typing import Any, Dict, List, Optional, Literal, Union
from dataclasses import dataclass

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
geno_each_sample = test_data_catalog.load("geno_each_sample")

In [ ]:
geno_each_sample

,rs77948203,rs1014626,rs9610458,rs5762201,rs1004237,rs134490,rs4821519,rs1003500,rs5756405
sample_id,,,,,,,,,
sample1,GG,TT,TT,GG,CC,NaN,GG,CC,AG
sample10,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample100,GG,TT,TT,GG,CC,TT,GG,CC,AG
sample1000,AG,TT,CT,GG,CC,CT,GG,CC,AA
sample1001,GG,TT,TT,GG,CC,CT,CG,CC,AG
...,...,...,...,...,...,...,...,...,...
sample995,GG,TT,CT,GG,CC,CT,CC,CC,AG
sample996,GG,TT,CT,GG,CC,TT,GG,CC,AG
sample997,GG,TT,CT,AG,CC,TT,GG,CC,AG


In [ ]:
#export

@dataclass
class RsidComboInfo():
    df: pd.DataFrame
    rsid_list: List[str]
    NA_val: str

    def query(self, **rsid_dict):
        if not all([rsid in self.rsid_list for rsid in rsid_dict.keys()]):
            raise ValueError("Some Rsid are not in the dataframe")
        filtered_df = self.df.copy()
        for rsid, geno in rsid_dict.items():
            filtered_df = filtered_df.query(f"{rsid} == '{geno}'")
        return filtered_df
    
    def get_all_genos(self, rsid:str):
        return self.df[rsid].unique()
    
    @property
    def total_samples_with_NA(self):
        return self.df.unique_samples_count.sum()
    
    @property
    def num_samples_NA(self):
        return self.df.loc[(self.df[self.rsid_list] == self.NA_val).any(axis=1)].unique_samples_count.sum()
    @property
    def total_samples_no_NA(self):
        return self.total_samples_with_NA  - self.num_samples_NA
    
    def __repr__(self):
        return f"{self.__class__}(rsid_list = {self.rsid_list}, NA_val = '{self.NA_val}')"
    
def get_geno_combination_df(geno_each_sample_df: pd.DataFrame, rsid_list:List[str], NA_val="NA", as_df= False):
    #geno_each_sample_df[["rs77948203", "rs1014626", "rs1004237"]].reset_index().groupby(["rs77948203", "rs1014626", "rs1004237"])["sample_id"].unique().reset_index()
    geno_combination_df = geno_each_sample_df[rsid_list].fillna(NA_val).reset_index().groupby(rsid_list)["sample_id"].agg(**{"unique_samples_id":"unique", "unique_samples_count": "nunique"}).reset_index()
    if as_df:
        return geno_combination_df
    else:
        info_obj = RsidComboInfo(df= geno_combination_df, rsid_list = rsid_list, NA_val= NA_val)
    return info_obj
    

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test_rsid_info_obj = get_geno_combination_df(geno_each_sample,["rs77948203"])
test_rsid_info_obj

<class '__main__.RsidComboInfo'>(rsid_list = ['rs77948203'], NA_val = 'NA')

In [ ]:
test_rsid_info_obj.df

,rs77948203,unique_samples_id,unique_samples_count
0,AA,"[sample1445, sample1545, sample2271, sample237...",46
1,AG,"[sample1000, sample1002, sample1025, sample102...",1227
2,GG,"[sample1, sample10, sample100, sample1001, sam...",8499


In [ ]:
assert test_rsid_info_obj.df.equals(geno_each_sample["rs77948203"].fillna("NA").reset_index().groupby("rs77948203")["sample_id"].agg(**{"unique_samples_id":"unique", "unique_samples_count": "nunique"}).reset_index())
assert test_rsid_info_obj.rsid_list == ['rs77948203']

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---

In [ ]:
get_geno_combination_df(geno_each_sample,["rs77948203"]).query(rs77948203='AA')

,rs77948203,unique_samples_id,unique_samples_count
0,AA,"[sample1445, sample1545, sample2271, sample237...",46


In [ ]:
get_geno_combination_df(geno_each_sample,["rs77948203", "rs1014626"]).df

,rs77948203,rs1014626,unique_samples_id,unique_samples_count
0,AA,TT,"[sample1445, sample1545, sample2271, sample237...",46
1,AG,CT,"[sample5663, sample5840]",2
2,AG,NA,[sample297],1
3,AG,TT,"[sample1000, sample1002, sample1025, sample102...",1224
4,GG,CT,"[sample1264, sample1359, sample1960, sample220...",26
5,GG,NA,"[sample1378, sample1496, sample2247, sample391...",8
6,GG,TT,"[sample1, sample10, sample100, sample1001, sam...",8465


You can query multiple genotypes by passing in keyword arguments or a dictionary

In [ ]:
query_result_1 = get_geno_combination_df(geno_each_sample,["rs77948203", "rs1014626"]).query(rs77948203= 'AG', rs1014626= 'TT')
query_result_1

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rs77948203,rs1014626,unique_samples_id,unique_samples_count
3,AG,TT,"[sample1000, sample1002, sample1025, sample102...",1224


In [ ]:
query_result_2 = get_geno_combination_df(geno_each_sample,["rs77948203", "rs1014626"]).query(**{"rs77948203": 'AG', "rs1014626": 'TT'})
query_result_2

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rs77948203,rs1014626,unique_samples_id,unique_samples_count
3,AG,TT,"[sample1000, sample1002, sample1025, sample102...",1224


In [ ]:
assert query_result_1.equals(query_result_2)

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Querying an unknown rsid will lead to an error

In [ ]:
with ExceptionExpected(ex=ValueError, regex = "Some Rsid are not in the dataframe"):test_rsid_info_obj.query(rs7794820='AA') #missing the last digit of rsid

In [ ]:
get_geno_combination_df(geno_each_sample,["rs77948203"]).query(rs77948203='NA').empty

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True

You can choose to return a simple dataframe, or an object that has enhanced capabilities

In [ ]:
get_geno_combination_df(geno_each_sample,["rs1014626"]).df

,rs1014626,unique_samples_id,unique_samples_count
0,CT,"[sample1264, sample1359, sample1960, sample220...",28
1,NA,"[sample1378, sample1496, sample2247, sample297...",9
2,TT,"[sample1, sample10, sample100, sample1000, sam...",9735


In [ ]:
get_geno_combination_df(geno_each_sample,["rs1014626"], as_df = True)

,rs1014626,unique_samples_id,unique_samples_count
0,CT,"[sample1264, sample1359, sample1960, sample220...",28
1,NA,"[sample1378, sample1496, sample2247, sample297...",9
2,TT,"[sample1, sample10, sample100, sample1000, sam...",9735


In [ ]:
assert get_geno_combination_df(geno_each_sample,["rs1014626"]).df.equals(get_geno_combination_df(geno_each_sample,["rs1014626"], as_df = True))

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
test_geno_combination_df = get_geno_combination_df(geno_each_sample,["rs77948203", "rs1014626"])
test_geno_combination_df.df

,rs77948203,rs1014626,unique_samples_id,unique_samples_count
0,AA,TT,"[sample1445, sample1545, sample2271, sample237...",46
1,AG,CT,"[sample5663, sample5840]",2
2,AG,NA,[sample297],1
3,AG,TT,"[sample1000, sample1002, sample1025, sample102...",1224
4,GG,CT,"[sample1264, sample1359, sample1960, sample220...",26
5,GG,NA,"[sample1378, sample1496, sample2247, sample391...",8
6,GG,TT,"[sample1, sample10, sample100, sample1001, sam...",8465


In [ ]:
test_geno_combination_df.num_samples_NA

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


9

In [ ]:
NA_samples_df = test_geno_combination_df.df.loc[(test_geno_combination_df.df[["rs77948203", "rs1014626"]] == "NA").any(axis=1)]
NA_samples_df

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rs77948203,rs1014626,unique_samples_id,unique_samples_count
2,AG,NA,[sample297],1
5,GG,NA,"[sample1378, sample1496, sample2247, sample391...",8


In [ ]:
assert test_geno_combination_df.num_samples_NA == NA_samples_df["unique_samples_count"].sum() == 9
assert test_geno_combination_df.total_samples_with_NA == 9772
assert test_geno_combination_df.total_samples_no_NA == 9763

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
get_geno_combination_df(geno_each_sample,["rs77948203", "rs1014626", "rs134490"])

/Users/ahoang/Documents/Learning/nbdev_tutorial_2nd_try/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,rs77948203,rs1014626,rs134490,unique_samples_id,unique_samples_count
0,AA,TT,CT,"[sample1445, sample1545, sample2271, sample281...",13
1,AA,TT,NA,"[sample525, sample7180, sample926, sample9293]",4
2,AA,TT,TT,"[sample2376, sample2519, sample2963, sample311...",29
3,AG,CT,CT,[sample5663],1
4,AG,CT,NA,[sample5840],1
5,AG,NA,CT,[sample297],1
6,AG,TT,CC,"[sample1747, sample1957, sample2090, sample216...",24
7,AG,TT,CT,"[sample1000, sample1002, sample1040, sample104...",312
8,AG,TT,NA,"[sample1035, sample1094, sample1098, sample110...",157
9,AG,TT,TT,"[sample1025, sample1028, sample1038, sample104...",731
